In [1]:
from Algorithms import LinkMomentum
from BacktestMethods import ResearchTest
from Display import BacktestDisplay
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def mlink_momentum_test(dir_list, corr, path, interval=250, alpha_val=False, signal_thresholds=[],num_terms=10):
    op = []
    df = []
    ticks = []
    algs = []
    arrs = []

    sp500 = pd.read_pickle(path+'SP500Open.pkl')
    for d in dir_list:
            open_data = pd.DataFrame()
            close_data = pd.DataFrame()

            # open the data for the correlation matrix
            with open(path+d+corr+"/"+str(interval)+"Day.pkl", 'rb') as handle:
                arr=pd.read_pickle(handle)

            open_data=pd.read_pickle(path+d+'Open.pkl') # load Open Data
            close_data=pd.read_pickle(path+d+'Close.pkl') # load Close Data

            # get the first, last keys (dates) and the indices for corresponding keys
            first_key=list(arr.keys())[0]
            last_key=list(arr.keys())[-1]
            start_index=list(str(d.date()) for d in close_data.index).index(first_key)
            end_index=list(str(d.date()) for d in close_data.index).index(last_key)


            t=list(i for i in open_data.columns[:int(len(open_data.columns)/2)]) # stock tickers to look at

            start_date=list(d for d in open_data.index)[start_index] # first date of trading
            end_date=list(d for d in open_data.index)[end_index] # last date of trading

            dates = list(x.date() for x in close_data.index)[start_index+20:-20]

            ticks.append(t)
            op.append(open_data)
            df.append(close_data)
            arrs.append(arr)
            algs.append(LinkMomentum(arr, t, k2=1,method="dif", alpha_val=alpha_val, num_terms=num_terms))

    # print("Start Date: " + str(dates[0]) + " --- End Date: " + str(dates[-1]))
    bt = ResearchTest(algs,df,op,tickers=ticks,start_date=start_date,end_date=end_date,dates=dates,meas_stock=sp500, multi_algs=True, signal_thresholds=False) # initialize backtest

    bt.backtest() # backtest
    bt.plot("Combined")
    # e = bt.eval()

    with open("log.txt", "w") as f:
        f.write(bt.log)

    # print(dir_list[0][:-1] + ". Buy: " + str(bt.avg_bsignal[0]) + " --- Sell: " + str(bt.avg_ssignal[0]))
    return bt.calc_vol(), bt.sharpe(), bt.comp_annual(), 0

In [ ]:
a = 0.75
index_list = ["Electric/", "Energy/", "Finance/", "Health/", "Indust/", "REstate/", "Tech/"]
avg_cag = 0
avg_sh = 0
num_terms = 1
for index in index_list:
    v, sh, cag, e = mlink_momentum_test([index], "Spearman", "TradingBot/Data/",  alpha_val = a, num_terms=num_terms, interval=250)
    print("Alpha: " + str(a) + " --- Terms: " + str(num_terms) + " --- Sharpe: " + str(round(sh,2)) + " --- CAG: " + str(round(cag,2)))